In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cd /content/drive/MyDrive/11777-Project-xs

In [2]:
import numpy as np
import scipy as sp
from tqdm import tqdm
import torch
from sklearn.model_selection import train_test_split

import sys
sys.path.append('/content/drive/MyDrive/11777-Project-xs/LGG/')
from prepare_data_DEAP import PrepareData
from cross_validation import CrossValidation
from train_model import *

In [3]:
# Define default parameters
default_args = {
    'dataset': 'DEAP',
    'data_path': './data_preprocessed_python/',
    'subjects': 22,
    'num_class': 2,
    'label_type': 'A',
    'segment': 4,
    'overlap': 0,
    'sampling_rate': 128,
    'scale_coefficient': 1,
    'input_shape': (1, 32, 512),
    'data_format': 'eeg',
    'random_seed': 2021,
    'max_epoch': 200,
    'patient': 20,
    'patient_cmb': 8,
    'max_epoch_cmb': 20,
    'batch_size': 64,
    'learning_rate': 1e-3,
    'step_size': 5,
    'dropout': 0.5,
    'LS': True,
    'LS_rate': 0.1,
    'save_path': '/content/drive/MyDrive/11777-Project-xs/LGG/save/',
    'load_path': '/content/drive/MyDrive/11777-Project-xs/LGG/save/max-acc.pth',
    'load_path_final': '/content/drive/MyDrive/11777-Project-xs/LGG/save/final_model.pth',
    'gpu': '0',
    'save_model': True,
    'model': 'LGGNet',
    'pool': 16,
    'pool_step_rate': 0.25,
    'T': 64,
    'graph_type': 'hem',
    'hidden': 32
}

In [4]:
######## Reproduce the result using the saved model ######
# parser.add_argument('--reproduce', action='store_true')
# args = parser.parse_args()
class Args:
    def __init__(self, param_dict):
        for key, value in param_dict.items():
            setattr(self, key, value)

default_args['data_path'] = '/content/drive/MyDrive/11777-Project-xs/data_preprocessed_python/'
default_args['reproduce'] = 'store_true'
# Create an Args object with default parameters
args = Args(default_args)


In [5]:
# only need to run once
sub_to_run = np.arange(args.subjects)
prepdt = PrepareData(args)
prepdt.run(sub_to_run, split=True, expand=True)

data:(40, 32, 7680) label:(40, 4)
Binary label generated!
The data and label are split: Data shape:(40, 15, 1, 32, 512) Label:(40, 15)
Data and label prepared!
data:(40, 15, 1, 32, 512) label:(40, 15)
----------------------
data:(40, 32, 7680) label:(40, 4)
Binary label generated!
The data and label are split: Data shape:(40, 15, 1, 32, 512) Label:(40, 15)
Data and label prepared!
data:(40, 15, 1, 32, 512) label:(40, 15)
----------------------
data:(40, 32, 7680) label:(40, 4)
Binary label generated!
The data and label are split: Data shape:(40, 15, 1, 32, 512) Label:(40, 15)
Data and label prepared!
data:(40, 15, 1, 32, 512) label:(40, 15)
----------------------
data:(40, 32, 7680) label:(40, 4)
Binary label generated!
The data and label are split: Data shape:(40, 15, 1, 32, 512) Label:(40, 15)
Data and label prepared!
data:(40, 15, 1, 32, 512) label:(40, 15)
----------------------
data:(40, 32, 7680) label:(40, 4)
Binary label generated!
The data and label are split: Data shape:(40, 

### train

In [7]:
from utils import *

def test(args, data, label, reproduce, subject, fold, path=None):
    set_up(args)
    seed_all(args.random_seed)
    test_loader = get_dataloader(data, label, args.batch_size)

    model = get_model(args)
    if CUDA:
        model = model.cuda()
    loss_fn = nn.CrossEntropyLoss()
    print('myTest')

    if reproduce:
        if path is None:
            model_name_reproduce = 'sub' + str(subject) + '.pth'
            data_type = 'model_{}_{}'.format(args.data_format, args.label_type)
            experiment_setting = 'T_{}_pool_{}'.format(args.T, args.pool)
            load_path_final = os.path.join(args.save_path, experiment_setting, data_type, model_name_reproduce)
            model.load_state_dict(torch.load(load_path_final))
        else:
            model.load_state_dict(torch.load(path))
    else:
        model.load_state_dict(torch.load(args.load_path_final))
    loss, pred, act = predict(
        data_loader=test_loader, net=model, loss_fn=loss_fn
    )
    acc, f1, cm = get_metrics(y_pred=pred, y_true=act)
    print('>>> Test:  loss={:.4f} acc={:.4f} f1={:.4f}'.format(loss, acc, f1))
    return acc, pred, act


# train user-model

In [8]:
import os
data_save_folder = '/content/drive/MyDrive/11777-Project-xs/data_face_seg_mean'
face_embed_file_list = os.listdir(data_save_folder)
# print(len(face_embed_file_list))
face_embed_file_list = [e for e in face_embed_file_list if 'embed' in e]
sub_trial_list = {}
for file in face_embed_file_list:
    # print(file)
    sub, trial, _, _, _ = file.split('_')
    sub, trial = int(sub[1:]), int(trial[5:])
    if sub in sub_trial_list.keys():
        sub_trial_list[sub].append(trial)
    else:
        sub_trial_list[sub] = [trial]

for sub in range(1,23):
    print('sub {}, {} trials with facial video'.format(sub, len(sub_trial_list[sub])))

sub 1, 40 trials with facial video
sub 2, 40 trials with facial video
sub 3, 39 trials with facial video
sub 4, 40 trials with facial video
sub 5, 39 trials with facial video
sub 6, 40 trials with facial video
sub 7, 40 trials with facial video
sub 8, 40 trials with facial video
sub 9, 40 trials with facial video
sub 10, 40 trials with facial video
sub 11, 37 trials with facial video
sub 12, 40 trials with facial video
sub 13, 40 trials with facial video
sub 14, 39 trials with facial video
sub 15, 40 trials with facial video
sub 16, 40 trials with facial video
sub 17, 40 trials with facial video
sub 18, 40 trials with facial video
sub 19, 40 trials with facial video
sub 20, 40 trials with facial video
sub 21, 40 trials with facial video
sub 22, 40 trials with facial video


In [10]:
CUDA = torch.cuda.is_available()
fold = 'noFold'

for sub in range(22):
    print('='*48)
    cv = CrossValidation(args)
    data, label = cv.load_per_subject(sub)
    trial_to_remove = [i-1 for i in range(1,41) if i not in sub_trial_list[sub+1]]

    if len(trial_to_remove) > 0:
        mask = np.ones(data.shape[0], dtype=bool)
        mask[trial_to_remove] = False
        data = data[mask]
        label = label[mask]
    print(sub+1, trial_to_remove, data.shape[0])
    trial_id = np.arange(data.shape[0])
    trial_id_train, trial_id_test = train_test_split(trial_id, test_size=0.2, random_state=42)
    trial_id_tra, trial_id_val = train_test_split(trial_id_train, test_size=0.2, random_state=42)
    print(trial_id_tra.shape[0], trial_id_val.shape[0], trial_id_test.shape[0])

    data_train, data_val, label_train, label_val = \
        data[trial_id_tra], data[trial_id_val], label[trial_id_tra], label[trial_id_val]
    data_train, data_val, label_train, label_val = \
        np.concatenate(data_train, axis=0), np.concatenate(data_val, axis=0), np.concatenate(label_train, axis=0), np.concatenate(label_val, axis=0)
    print(data_train.shape, data_val.shape, label_train.shape, label_val.shape)

    data_test, label_test = \
        data[trial_id_test], label[trial_id_test]
    data_test, label_test = \
        np.concatenate(data_test, axis=0), np.concatenate(label_test, axis=0)
    print(data_test.shape, label_test.shape)

    data_train = torch.from_numpy(data_train).float()
    label_train = torch.from_numpy(label_train).long()
    data_val = torch.from_numpy(data_val).float()
    label_val = torch.from_numpy(label_val).long()
    data_test = torch.from_numpy(data_test).float()
    label_test = torch.from_numpy(label_test).long()

    acc_val, F1_val = train(args=args,
                        data_train=data_train,
                        label_train=label_train,
                        data_val=data_val,
                        label_val=label_val,
                        subject = str(sub+1),
                        fold='noFold')
    model = get_model(args)
    if CUDA:
        model = model.cuda()
    model.load_state_dict(torch.load(os.path.join(args.save_path, 'candidate.pth')))

    model_name_reproduce = 'sub' + str(sub+1) + '.pth'
    data_type = 'model_{}_{}'.format(args.data_format, args.label_type)
    experiment_setting = 'T_{}_pool_{}'.format(args.T, args.pool)
    save_path = os.path.join(args.save_path, experiment_setting, data_type)
    ensure_path(save_path)
    model_name_reproduce = os.path.join(save_path, model_name_reproduce)
    print(model_name_reproduce)
    torch.save(model.state_dict(), model_name_reproduce)

    model.load_state_dict(torch.load(model_name_reproduce))
    acc_test, pred, act = test(args=args, data=data_test, label=label_test,
                                               reproduce=args.reproduce,
                                               subject=str(sub+1), fold='noFold')


>>> Data:(40, 15, 1, 32, 512) Label:(40, 15)
1 [] 40
25 7 8
(375, 1, 32, 512) (105, 1, 32, 512) (375,) (105,)
(120, 1, 32, 512) (120,)
using gpu: 0
epoch 1, loss=1.4168 acc=0.5013 f1=0.5405
epoch 1, val, loss=0.6899 acc=0.5714 f1=0.7273
ETA:9s/31m SUB:1 FOLD:noFold
epoch 2, loss=0.7365 acc=0.6240 f1=0.7432
epoch 2, val, loss=0.6862 acc=0.5714 f1=0.7273
ETA:10s/18m SUB:1 FOLD:noFold
epoch 3, loss=0.6262 acc=0.6987 f1=0.7426
epoch 3, val, loss=0.6896 acc=0.5714 f1=0.7273
ETA:11s/13m SUB:1 FOLD:noFold
epoch 4, loss=0.6088 acc=0.7040 f1=0.7633
epoch 4, val, loss=0.6897 acc=0.5714 f1=0.7273
ETA:13s/11m SUB:1 FOLD:noFold
epoch 5, loss=0.5158 acc=0.7840 f1=0.8350
epoch 5, val, loss=0.6952 acc=0.5714 f1=0.7273
ETA:14s/10m SUB:1 FOLD:noFold
epoch 6, loss=0.4333 acc=0.8747 f1=0.8962
epoch 6, val, loss=0.6970 acc=0.5714 f1=0.7273
ETA:15s/9m SUB:1 FOLD:noFold
epoch 7, loss=0.3747 acc=0.9067 f1=0.9234
epoch 7, val, loss=0.7142 acc=0.5714 f1=0.7273
ETA:16s/8m SUB:1 FOLD:noFold
epoch 8, loss=0.3029 a

In [ ]:

type(data_test)

numpy.ndarray

In [ ]:
path = '/content/drive/MyDrive/11777-Project-xs/LGG/save/candidate.pth'